In [ ]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets
from torch import nn, optim, autograd
import pdb
from tqdm import tqdm
import pickle

In [ ]:
parser = argparse.ArgumentParser(description='Colored MNIST')
parser.add_argument('--hidden_dim', type=int, default=390)
parser.add_argument('--l2_regularizer_weight', type=float,default=0.00110794568)
parser.add_argument('--lr', type=float, default=0.0004898536566546834)
parser.add_argument('--n_restarts', type=int, default=3)
parser.add_argument('--penalty_anneal_iters', type=int, default=190)
parser.add_argument('--penalty_weight', type=float, default=91257.18613115903)
parser.add_argument('--steps', type=int, default=901)
parser.add_argument('--grayscale_model', type=int, default=0)
parser.add_argument('--eiil', type=int, default=1)
parser.add_argument('--reiil_iters', type=int, default=10)
flags = parser.parse_args(['--hidden_dim', '390'])
torch.cuda.set_device(0)

In [ ]:
print('Flags:')
for k,v in sorted(vars(flags).items()):
  print("\t{}: {}".format(k, v))

In [ ]:
def split_data_opt(envs, model, n_steps=5000, n_samples=-1):
  """Learn soft environment assignment."""
  images = torch.cat((envs[0]['images'],envs[1]['images']),0)
  labels = torch.cat((envs[0]['labels'],envs[1]['labels']),0)
  orig_labels = torch.cat((envs[0]['orig_labels'],envs[1]['orig_labels']),0)
  print('size of pooled envs: '+str(len(images)))

  scale = torch.tensor(1.).cuda().requires_grad_()
  logits = model(images)
  loss = nll(logits * scale, labels, reduction='none')

  env_w = torch.randn(len(logits)).cuda().requires_grad_()
  optimizer = optim.Adam([env_w], lr=0.01)

  print('learning soft environment assignments')
  prev_penalty = 0
  ind = 0
  max_diff = -np.inf
  pbar = tqdm(range(n_steps))
  for i in pbar:
    # penalty for env a
    lossa = (loss.squeeze() * env_w.sigmoid()).mean()
    grada = autograd.grad(lossa, [scale], create_graph=True)[0]
    penaltya = torch.sum(grada**2)
    # penalty for env b
    lossb = (loss.squeeze() * (1-env_w.sigmoid())).mean()
    gradb = autograd.grad(lossb, [scale], create_graph=True)[0]
    penaltyb = torch.sum(gradb**2)
    # negate
    npenalty = - torch.stack([penaltya, penaltyb]).mean()

    if i > 0:
      diff = abs(npenalty.item() - prev_penalty)
    else:
      diff = -np.inf
    if diff > max_diff:
      max_diff = diff
      ind = i
    pbar.set_description_str(desc='Negative Penalty: '+str(npenalty.item())+', Diff: '+str(diff)+', Max Diff: '+str(max_diff)+'('+str(ind)+')')
    prev_penalty = npenalty.item()

    optimizer.zero_grad()
    npenalty.backward(retain_graph=True)
    optimizer.step()

  # split envs based on env_w threshold
  new_envs = []

  idx0 = (env_w.sigmoid()>.5)
  idx1 = (env_w.sigmoid()<=.5)

  mid = len(idx0) // 2
  c1 = torch.count_nonzero(idx0[:mid])
  c2 = torch.count_nonzero(idx1[mid:])
  acc = (c1 + c2) / len(idx0)

  if acc < 0.5:
    acc = 1 - acc

  print (f'Environment assignment accuracy: {acc}')

  # train envs
  for idx in (idx0, idx1):
    new_envs.append(dict(images=images[idx], labels=labels[idx], orig_labels=orig_labels[idx]))
  # test env
  new_envs.append(dict(images=envs[-1]['images'],
                       labels=envs[-1]['labels'],
                       orig_labels=envs[-1]['orig_labels']))
  print('size of env0: '+str(len(new_envs[0]['images'])))
  print('size of env1: '+str(len(new_envs[1]['images'])))
  print('size of env2: '+str(len(new_envs[2]['images'])))
  return new_envs, env_w

In [ ]:
final_train_accs_erm = []
final_train_accs_erm2 = []
final_train_accs_irm = []
final_train_accs_erm_rnd = []
final_train_accs_eiil_irm = []
final_train_accs_eiil_erm = []
final_train_accs_eiil_erm2 = []
final_train_accs_ei_werm = []
final_test_accs_erm = []
final_test_accs_erm2 = []
final_test_accs_irm = []
final_test_accs_erm_rnd = []
final_test_accs_eiil_irm = []
final_test_accs_eiil_erm = []
final_test_accs_eiil_erm2 = []
final_test_accs_ei_werm = []

In [ ]:
for restart in range(flags.n_restarts):
  print("Restart", restart)

  # Load MNIST, make train/val splits, and shuffle train set examples

  mnist = datasets.MNIST('~/datasets/mnist', train=True, download=True)
  mnist_train = (mnist.data[:50000], mnist.targets[:50000])
  mnist_val = (mnist.data[50000:], mnist.targets[50000:])

  rng_state = np.random.get_state()
  np.random.shuffle(mnist_train[0].numpy())
  np.random.set_state(rng_state)
  np.random.shuffle(mnist_train[1].numpy())

  # Build environments

  def make_environment(images, labels, e):

    def torch_bernoulli(p, size):
      return (torch.rand(size) < p).float()
    def torch_xor(a, b):
      return (a-b).abs() # Assumes both inputs are either 0 or 1
    # 2x subsample for computational convenience
    images = images.reshape((-1, 28, 28))[:, ::2, ::2]
    # Assign a binary label based on the digit; flip label with probability 0.25
    orig_labels = (labels < 5).float()
    synthetic_labels = torch_xor(orig_labels, torch_bernoulli(0.25, len(orig_labels)))
    # Assign a color based on the label; flip the color with probability e
    colors = torch_xor(synthetic_labels, torch_bernoulli(e, len(synthetic_labels)))

    flipped = (orig_labels != colors).nonzero(as_tuple=True)[0]

    # Apply the color to the image by zeroing out the other color channel
    images = torch.stack([images, images], dim=1)
    images[torch.tensor(range(len(images))), (1-colors).long(), :, :] *= 0
    return {
      'images': (images.float() / 255.).cuda(),
      'labels': synthetic_labels[:, None].cuda(),
      'orig_labels': labels[:, None].float().cuda(),
      'flipped': flipped.cuda()
    }
  envs = [
    make_environment(mnist_train[0][::2], mnist_train[1][::2], 0.2),
    make_environment(mnist_train[0][1::2], mnist_train[1][1::2], 0.1),
    make_environment(mnist_val[0], mnist_val[1], 0.9)
  ]

  flipped = []
  for env in envs:
    flipped.append(env['flipped'])

  flipped = torch.cat((flipped[0], flipped[1] + 25000))

  # Define and instantiate the model

  class MLP(nn.Module):
    def __init__(self):
      super(MLP, self).__init__()
      if flags.grayscale_model:
        lin1 = nn.Linear(14 * 14, flags.hidden_dim)
      else:
        lin1 = nn.Linear(2 * 14 * 14, flags.hidden_dim)
      lin2 = nn.Linear(flags.hidden_dim, flags.hidden_dim)
      lin3 = nn.Linear(flags.hidden_dim, 1)
      for lin in [lin1, lin2, lin3]:
        nn.init.xavier_uniform_(lin.weight)
        nn.init.zeros_(lin.bias)
      self._main = nn.Sequential(lin1, nn.ReLU(True), lin2, nn.ReLU(True), lin3)
    def forward(self, input):
      if flags.grayscale_model:
        out = torch.reshape(input, (input.shape[0], 2, 14 * 14)).sum(dim=1)
      else:
        out = torch.reshape(input, (input.shape[0], 2 * 14 * 14))
      out = self._main(out)
      return out

  # Define loss function helpers

  def nll(logits, y, reduction='mean'):
    return nn.functional.binary_cross_entropy_with_logits(logits, y, reduction=reduction)

  def mean_accuracy(logits, y, reduce='sum'):
    preds = (logits > 0.).float()
    if reduce == 'mean':
      return ((preds - y).abs() < 1e-2).float().mean()
    else:
      return ((preds - y).abs() < 1e-2).float().sum()

  def penalty(logits, y):
    scale = torch.tensor(1.).cuda().requires_grad_()
    loss = nll(logits * scale, y)
    grad = autograd.grad(loss, [scale], create_graph=True)[0]
    return torch.sum(grad**2)

  # Train loop

  def pretty_print(*values):
    col_width = 13
    def format_val(v):
      if not isinstance(v, str):
        v = np.array2string(v, precision=5, floatmode='fixed')
      return v.ljust(col_width)
    str_values = [format_val(v) for v in values]
    print("   ".join(str_values))

  mlp_irm = MLP().cuda()

  optimizer_irm = optim.Adam(mlp_irm.parameters(), lr=flags.lr)

  if flags.eiil:

    print ('IRM')
    pretty_print('step', 'train nll', 'train acc', 'train penalty', 'test acc')

    # Pre-train IRM reference model
    for step in range(flags.steps):
      for env in envs:
        logits = mlp_irm(env['images'])
        env['nll'] = nll(logits, env['labels'], reduction='sum')
        env['acc'] = mean_accuracy(logits, env['labels'])
        env['penalty'] = penalty(logits, env['labels'])

      tot_samples = len(envs[0]['images']) + len(envs[1]['images'])
      train_nll = torch.stack([envs[0]['nll'], envs[1]['nll']]).sum() / tot_samples
      train_acc = torch.stack([envs[0]['acc'], envs[1]['acc']]).sum() / tot_samples
      train_penalty = torch.stack([envs[0]['penalty'], envs[1]['penalty']]).mean()

      weight_norm = torch.tensor(0.).cuda()
      for w in mlp_irm.parameters():
        weight_norm += w.norm().pow(2)

      loss = train_nll.clone()
      loss += flags.l2_regularizer_weight * weight_norm
      # NOTE: IRM penalties used for testing.
      penalty_weight = (flags.penalty_weight
       if step >= flags.penalty_anneal_iters else 1.0)
      loss += penalty_weight * train_penalty
      if penalty_weight > 1.0:
       # Rescale the entire loss to keep gradients in a reasonable range
       loss /= penalty_weight

      optimizer_irm.zero_grad()
      loss.backward()
      optimizer_irm.step()

      test_acc = envs[2]['acc'] / len(envs[2]['images'])
      if step % 100 == 0:
        pretty_print(
          np.int32(step),
          train_nll.detach().cpu().numpy(),
          train_acc.detach().cpu().numpy(),
          train_penalty.detach().cpu().numpy(),
          test_acc.detach().cpu().numpy()
        )

    final_train_accs_irm.append(train_acc.detach().cpu().numpy())
    final_test_accs_irm.append(test_acc.detach().cpu().numpy())
        
    min_env = None
    print ('Starting fresh train for EIILv2..')
    new_envs = envs.copy()
    for eiil_ind in range(flags.reiil_iters):
      mlp_erm = MLP().cuda()
      
      print ('ERM')
      optimizer_erm = optim.Adam(mlp_erm.parameters(), lr=flags.lr)
      # First ERM is on the whole training dataset.
      if eiil_ind == 0:
        pretty_print('step', 'train nll', 'train acc', 'train penalty', 'test acc')
      else:
        pretty_print('step', 'train nll', 'train acc (0)', 'train acc (1)', 'train penalty', 'test acc')

      for step in range(flags.steps):
        for env in new_envs:
          logits = mlp_erm(env['images'])
          env['nll'] = nll(logits, env['labels'], reduction='sum')
          env['acc'] = mean_accuracy(logits, env['labels'])

        if min_env:
          mlp_erm.eval()
          logits = mlp_erm(min_env['images'])
          min_env['acc'] = mean_accuracy(logits, min_env['labels'])
          mlp_erm.train()

        if eiil_ind == 0:
          tot_samples = len(new_envs[0]['images']) + len(new_envs[1]['images'])
          train_nll = torch.stack([new_envs[0]['nll'], new_envs[1]['nll']]).sum() / tot_samples
          train_acc = torch.stack([new_envs[0]['acc'], new_envs[1]['acc']]).sum() / tot_samples
        else:
          tot_samples = len(new_envs[0]['images'])
          train_nll = new_envs[0]['nll'].sum() / tot_samples
          train_acc = new_envs[0]['acc'].sum() / tot_samples
          min_env_acc = min_env['acc'].sum() / len(min_env['images'])

        weight_norm = torch.tensor(0.).cuda()
        for w in mlp_erm.parameters():
          weight_norm += w.norm().pow(2)

        loss = train_nll.clone()
        loss += flags.l2_regularizer_weight * weight_norm
      
        optimizer_erm.zero_grad()
        loss.backward()
        optimizer_erm.step()

        if eiil_ind == 0:
          test_acc = new_envs[2]['acc'] / len(new_envs[2]['images'])
          if step % 100 == 0:
            pretty_print(np.int32(step), train_nll.detach().cpu().numpy(), train_acc.detach().cpu().numpy(), train_penalty.detach().cpu().numpy(), test_acc.detach().cpu().numpy())
        else:
          test_acc = new_envs[1]['acc'] / len(new_envs[1]['images'])
          if step % 100 == 0:
            pretty_print(np.int32(step), train_nll.detach().cpu().numpy(), train_acc.detach().cpu().numpy(), min_env_acc.detach().cpu().numpy(), train_penalty.detach().cpu().numpy(), test_acc.detach().cpu().numpy())

      # Split environments
      new_envs, env_w = split_data_opt(envs, mlp_erm)

      rest_envs = new_envs.copy()
      
      if eiil_ind < flags.reiil_iters - 1:
        maj_ind = 0
        if len(new_envs[1]['images']) > len(new_envs[0]['images']):
          maj_ind = 1
        
        min_env = new_envs[1 - maj_ind]
        new_envs = [new_envs[maj_ind], new_envs[-1]]
      
      all_min = env_w.sigmoid() <= 0.5
      if maj_ind == 1:
        all_min = env_w.sigmoid() > 0.5

      count_min = torch.count_nonzero(all_min[flipped])
      print (f'Total flipped in minority environment: {count_min} / {len(flipped)} ({count_min / len(flipped)})%')

      mlp = MLP().cuda()
      optimizer = optim.Adam(mlp.parameters(), lr=flags.lr)
      print ('REI_WERM')
      pretty_print('step', 'train nll', 'train acc', 'train penalty', 'test acc')
      for step in range(flags.steps):
        for env in rest_envs:
          logits = mlp(env['images'])
          env['nll'] = nll(logits, env['labels'])
          env['acc'] = mean_accuracy(logits, env['labels'])
          env['penalty'] = penalty(logits, env['labels'])

        tot_samples = len(rest_envs[0]['images']) + len(rest_envs[1]['images'])

        train_nll = torch.stack([rest_envs[0]['nll'], rest_envs[1]['nll']]).mean()
        train_acc = torch.stack([rest_envs[0]['acc'], rest_envs[1]['acc']]).sum() / tot_samples
        train_penalty = torch.stack([rest_envs[0]['penalty'], rest_envs[1]['penalty']]).mean()

        weight_norm = torch.tensor(0.).cuda()
        for w in mlp.parameters():
          weight_norm += w.norm().pow(2)

        loss = train_nll.clone()
        loss += flags.l2_regularizer_weight * weight_norm

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        test_acc = rest_envs[2]['acc'] / len(rest_envs[2]['images'])
        if step % 100 == 0:
          pretty_print(
            np.int32(step),
            train_nll.detach().cpu().numpy(),
            train_acc.detach().cpu().numpy(),
            train_penalty.detach().cpu().numpy(),
            test_acc.detach().cpu().numpy()
          )

      final_train_accs_ei_werm.append(train_acc.detach().cpu().numpy())
      final_test_accs_ei_werm.append(test_acc.detach().cpu().numpy())
      print('Final train acc (mean/std across restarts so far):')
      print(np.mean(final_train_accs_ei_werm), np.std(final_train_accs_ei_werm))
      print('Final test acc (mean/std across restarts so far):')
      print(np.mean(final_test_accs_ei_werm), np.std(final_test_accs_ei_werm))

      mlp = MLP().cuda()
      optimizer = optim.Adam(mlp.parameters(), lr=flags.lr)
      print ('EIIL')
      pretty_print('step', 'train nll', 'train acc', 'train penalty', 'test acc')
      for step in range(flags.steps):
        for env in rest_envs:
          logits = mlp(env['images'])
          # env['nll'] = nll(logits, env['labels'], reduction='sum')
          env['nll'] = nll(logits, env['labels'])
          env['acc'] = mean_accuracy(logits, env['labels'])
          env['penalty'] = penalty(logits, env['labels'])

        tot_samples = len(rest_envs[0]['images']) + len(rest_envs[1]['images'])
        # train_nll = torch.stack([rest_envs[0]['nll'], rest_envs[1]['nll']]).sum() / tot_samples
        train_nll = torch.stack([rest_envs[0]['nll'], rest_envs[1]['nll']]).mean()
        train_acc = torch.stack([rest_envs[0]['acc'], rest_envs[1]['acc']]).sum() / tot_samples
        train_penalty = torch.stack([rest_envs[0]['penalty'], rest_envs[1]['penalty']]).mean()

        weight_norm = torch.tensor(0.).cuda()
        for w in mlp.parameters():
          weight_norm += w.norm().pow(2)

        loss = train_nll.clone()
        loss += flags.l2_regularizer_weight * weight_norm
        penalty_weight = (flags.penalty_weight
            if step >= flags.penalty_anneal_iters else 1.0)
        loss += penalty_weight * train_penalty
        if penalty_weight > 1.0:
          # Rescale the entire loss to keep gradients in a reasonable range
          loss /= penalty_weight

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        test_acc = rest_envs[2]['acc'] / len(rest_envs[2]['images'])
        if step % 100 == 0:
          pretty_print(
            np.int32(step),
            train_nll.detach().cpu().numpy(),
            train_acc.detach().cpu().numpy(),
            train_penalty.detach().cpu().numpy(),
            test_acc.detach().cpu().numpy()
          )

      final_train_accs_eiil_erm2.append(train_acc.detach().cpu().numpy())
      final_test_accs_eiil_erm2.append(test_acc.detach().cpu().numpy())
      print('Final train acc (mean/std across restarts so far):')
      print(np.mean(final_train_accs_eiil_erm2), np.std(final_train_accs_eiil_erm2))
      print('Final test acc (mean/std across restarts so far):')
      print(np.mean(final_test_accs_eiil_erm2), np.std(final_test_accs_eiil_erm2))